In [25]:
suppressPackageStartupMessages(library(plyr))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(patchwork))
suppressPackageStartupMessages(library(stringr))
library(hise)
library(parallel)
library(introdataviz)

In [26]:
meta_data<-read.csv('/home//jupyter/BRI_Analysis/scRNA/hise_meta_data_2024-01-23_fixed.csv')
meta_data<-meta_data %>% filter(Covid_exlcusion=='no')
meta_data<-meta_data %>% filter(sample.visitName %in% c('Flu Year 1 Day 0','Flu Year 1 Day 90','Flu Year 2 Day 0','Flu Year 2 Day 90'))
donors<-names(table(meta_data$subject.subjectGuid)[table(meta_data$subject.subjectGuid)==4])
meta_data<-meta_data %>% filter(subject.subjectGuid %in% donors)

In [27]:
meta<-unique(meta_data[c('subject.subjectGuid','CMV.IgG.Serology.Result.Interpretation','subject.biologicalSex','cohort.cohortGuid')])

In [28]:
colnames(meta)<-c('donor','CMV','Sex','cohorts')

In [29]:
CV_Res_FILES<-list.files(path = "01A_CV_result/", pattern = "csv", all.files = TRUE,
           full.names = TRUE, recursive = FALSE,
           ignore.case = TRUE, include.dirs = TRUE, no.. = FALSE)

In [30]:
CV_Res_meta<-as.data.frame(CV_Res_FILES)

In [31]:
CV_Res_meta<-CV_Res_meta %>%
  mutate(    after_slash = str_extract(CV_Res_FILES, "(?<=//).+"),
    donor = str_extract(CV_Res_FILES, "(?<=//)[^_]+"),
    celltype = str_extract(after_slash, "(?<=_)[^.]+(?=\\.csv)") )%>%
  select(-after_slash)

In [32]:
CV_Res_meta<-left_join(CV_Res_meta,meta)

Joining with `by = join_by(donor)`


In [33]:
gene="HBB"
celltype="Core_naive_CD4_T_cell"
mean_expression_list<-mclapply(donors,function(i){  

print(i)
meta_data_donor<-meta_data %>% filter(subject.subjectGuid == i)
    
df_list<-mclapply(seq(from = 1, to = 4, by = 1),function(x){
df<-read.csv(paste0('01A_Mean_Expression_by_Celltype_and_Sample/',pull(meta_data_donor['pbmc_sample_id'])[x],'_AIFI_L3.csv'),row.names = 1,check.names = FALSE)
df<-as.data.frame(df)
colnames(df)<-paste0(pull(meta_data_donor['sample.visitName'])[x],':',colnames(df))   
return(df[c(gene),])
},mc.cores=4)

df_all<-do.call(cbind,df_list)
colnames(df_all)<-gsub(" ",'_',colnames(df_all))
df_all<-as.data.frame(t(df_all[, grepl(celltype, names(df_all), fixed = TRUE)]))
df_all$donor<-i
df_all$timepoint_celltype<-rownames(df_all)
rownames(df_all)<-NULL
return(df_all)
},mc.cores=30)

In [34]:
mean_df<-do.call(rbind,mean_expression_list)%>% 
  separate(timepoint_celltype, into = c("Flu_Year_Day", "Cell_Type"), sep = ":")

Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 71 rows [1, 2, 3, 4, 5,
6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”


In [35]:
colnames(mean_df)[1]<-"mean_expression"

In [36]:
mean_df$cohorts<-substr(mean_df$donor,1,3)


ERROR: Error in `$<-.data.frame`(`*tmp*`, cohorts, value = character(0)): replacement has 0 rows, data has 71


In [37]:
options(repr.plot.width = 8, repr.plot.height = 5)

ggplot(mean_df, aes(x=Flu_Year_Day, y=mean_expression, color=cohorts))  + 
  geom_boxplot() + geom_point()+
      stat_smooth(aes(group = donor), method = "lm",alpha=0.05, se = TRUE) +facet_wrap(~cohorts)+theme_bw(base_size=15)+ 
theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

ERROR while rich displaying an object: Error in `combine_vars()`:
! At least one layer must contain all faceting variables: `cohorts`
✖ Plot is missing `cohorts`
✖ Layer 1 is missing `cohorts`
✖ Layer 2 is missing `cohorts`
✖ Layer 3 is missing `cohorts`

Traceback:
1. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
2. tryCatchList(expr, classes, parentenv, handlers)
3. tryCatchOne(expr, names, parentenv, handlers[[1L]])
4. doTryCatch(return(expr), name, parentenv, handler)
5. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(N

In [38]:
options(repr.plot.width = 8, repr.plot.height = 5)

ggplot(mean_df, aes(x=Flu_Year_Day, y=mean_expression, color=cohorts))  + 
  geom_boxplot() + geom_point()+
      stat_smooth(aes(group = cohorts), method = "lm",alpha=0.05, se = TRUE) +facet_wrap(~cohorts)+theme_bw(base_size=15)+ 
theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))


ERROR while rich displaying an object: Error in `combine_vars()`:
! At least one layer must contain all faceting variables: `cohorts`
✖ Plot is missing `cohorts`
✖ Layer 1 is missing `cohorts`
✖ Layer 2 is missing `cohorts`
✖ Layer 3 is missing `cohorts`

Traceback:
1. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
2. tryCatchList(expr, classes, parentenv, handlers)
3. tryCatchOne(expr, names, parentenv, handlers[[1L]])
4. doTryCatch(return(expr), name, parentenv, handler)
5. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(N